In [1]:
from googletrans import Translator
import speech_recognition as sr
import subprocess
import os
from concurrent.futures import ThreadPoolExecutor
import subprocess
import os
from pydub import AudioSegment
import speech_recognition as sr
from googletrans import Translator
from moviepy.editor import VideoFileClip

In [2]:
# Extract audio from video using FFmpeg
def extract_audio(video_path, audio_path):
    subprocess.run(['ffmpeg', '-i', video_path, '-q:a',
                   '0', '-map', 'a', audio_path, '-n'], errors=None, capture_output=True)

# Segment the audio file


def save_segment(audio, i, segment_length_ms, dir_n):
    fn = f'{dir_n}/tmp{i}.wav'
    res = audio[i:i + segment_length_ms].export(fn)
    return fn


def segment_audio(audio_path, segment_length_ms):
    dir_n = audio_path.replace('.wav', '')
    if not os.path.exists(dir_n):
        os.mkdir(dir_n)
    with AudioSegment.from_file(audio_path) as audio:
        segments = [save_segment(audio, i, segment_length_ms, dir_n)
                    for i in range(0, len(audio), segment_length_ms)]
    return segments

# Transcribe audio segment to text


def transcribe_audio_segment(audio_segment):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_segment.export("temp.wav", format="wav")) as source:
        audio = recognizer.record(source)
    os.remove("temp.wav")
    try:
        transcription = recognizer.recognize_google(audio, language='zh-CN')
        return transcription
    except sr.UnknownValueError:
        return ""
    except sr.RequestError:
        return ""

# Translate text from Chinese to English


def translate_text(text, src_lang='zh-CN', dest_lang='en'):
    translator = Translator()
    translation = translator.translate(text, src=src_lang, dest=dest_lang)
    return translation.text

# Generate SRT file


def generate_srt(transcriptions, translations, srt_path, segment_length_ms):
    with open(srt_path, 'w') as f:
        for i, (transcription, translation) in enumerate(zip(transcriptions, translations)):
            start_time = i * segment_length_ms
            end_time = start_time + segment_length_ms
            start_time_str = format_time(start_time)
            end_time_str = format_time(end_time)
            f.write(f"{i + 1}\n")
            f.write(f"{start_time_str} --> {end_time_str}\n")
            f.write(f"{translation}\n\n")

# Format time in milliseconds to SRT time format


def format_time(ms):
    hours, ms = divmod(ms, 3600000)
    minutes, ms = divmod(ms, 60000)
    seconds, ms = divmod(ms, 1000)
    return f"{hours:02}:{minutes:02}:{seconds:02},{ms:03}"

In [20]:
import os
from pydub import AudioSegment
from pydub.exceptions import CouldntDecodeError
import speech_recognition as sr


def segment_audio(audio_path, segment_length_ms):
    dir_n = "audio_segments"
    if not os.path.exists(dir_n):
        os.mkdir(dir_n)

    try:
        audio = AudioSegment.from_file(audio_path)
    except CouldntDecodeError as e:
        raise RuntimeError(f"Failed to decode audio file: {e}")

    segments = []
    for i in range(0, len(audio), segment_length_ms):
        segment = audio[i:i+segment_length_ms]
        segment_path = os.path.join(
            dir_n, f"segment_{i//segment_length_ms}.wav")
        segment.export(segment_path, format="wav")
        segments.append(segment_path)

    return segments


def audio_segment_to_text(audio_path):
    # Initialize recognizer
    recognizer = sr.Recognizer()

    # Load the audio file
    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)

        # Convert audio to text
        try:
            text = recognizer.recognize_google(audio_data)
            return text
        except sr.UnknownValueError:
            return "Speech Recognition could not understand the audio"
        except sr.RequestError as e:
            return f"Could not request results from Google Speech Recognition service; {e}"


# Example usage
audio_path = "path/to/your/audio/file.mp3"  # Replace with your audio file path
segment_length_ms = 30000  # 30-second segments

# Ensure the file exists
if not os.path.exists(audio_path):
    raise FileNotFoundError(f"The file at path {audio_path} does not exist")

# Segment the audio file
try:
    segments = segment_audio(audio_path, segment_length_ms)
    for segment in segments:
        transcribed_text = audio_segment_to_text(segment)
        print(transcribed_text)
except RuntimeError as e:
    print(e)

FileNotFoundError: The file at path path/to/your/audio/file.mp3 does not exist

In [22]:
# Paths
video_path = files[10]
audio_path = video_path.replace('.mp4', '.wav')
srt_path = video_path.replace('.mp4', '.srt')
segment_length_ms = 30000  # 1/2 minute segments
# Segment the audio file
try:
    segments = segment_audio(audio_path, segment_length_ms)
    for segment in segments:
        transcribed_text = audio_segment_to_text(segment)
        print(transcribed_text)
except RuntimeError as e:
    print(e)

Failed to decode audio file: Decoding failed. ffmpeg returned error code: 183

Output from ffmpeg/avlib:

ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --ena

In [ ]:
transcribe_audio_segment(segments[0])

In [ ]:
# Paths
video_path = files[0]
audio_path = video_path.replace('.mp4', '.wav')
srt_path = video_path.replace('.mp4', '.srt')
segment_length_ms = 60000  # 1 minute segments

# Workflow
extract_audio(video_path, audio_path)
segments = segment_audio(audio_path, segment_length_ms)

transcriptions = [transcribe_audio_segment(segment) for segment in segments]
translations = [translate_text(transcription)
                for transcription in transcriptions]

generate_srt(transcriptions, translations, srt_path, segment_length_ms)

# Clean up
os.remove(audio_path)

print("Subtitle generation complete!")

In [17]:
s = segments[0]

NameError: name 'segments' is not defined

In [ ]:
recognizer = sr.Recognizer()
s.export('tmp.wav')

recognizer.listen(s)